In [55]:
import tensorflow as tf

import numpy as np
import os
import time

In [56]:
path_to_file = ('yemusic.txt')

In [57]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 266894 characters


In [58]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 266894 characters


In [59]:
# Take a look at the first 250 characters in text
print(text[:250])

Let the suicide doors up
I threw suicides on the tour bus
I threw suicides on the private jet
You know what that mean, I'm fly to death
I step in Def Jam buildin' like I'm the shit
Tell 'em give me fifty million or I'ma quit
Most rappers' taste


In [60]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

95 unique characters


In [68]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [69]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [70]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[57, 58, 59, 60, 61, 62, 63], [80, 81, 82]]>

In [71]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [72]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [73]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [74]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [75]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(266894,), dtype=int64, numpy=array([42, 61, 76, ..., 77, 16, 16], dtype=int64)>

In [76]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [77]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

L
e
t
 
t
h
e
 
s
u


In [78]:
seq_length = 100


In [82]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'L' b'e' b't' b' ' b't' b'h' b'e' b' ' b's' b'u' b'i' b'c' b'i' b'd'
 b'e' b' ' b'd' b'o' b'o' b'r' b's' b' ' b'u' b'p' b'\r' b'\n' b'I' b' '
 b't' b'h' b'r' b'e' b'w' b' ' b's' b'u' b'i' b'c' b'i' b'd' b'e' b's'
 b' ' b'o' b'n' b' ' b't' b'h' b'e' b' ' b't' b'o' b'u' b'r' b' ' b'b'
 b'u' b's' b'\r' b'\n' b'I' b' ' b't' b'h' b'r' b'e' b'w' b' ' b's' b'u'
 b'i' b'c' b'i' b'd' b'e' b's' b' ' b'o' b'n' b' ' b't' b'h' b'e' b' '
 b'p' b'r' b'i' b'v' b'a' b't' b'e' b' ' b'j' b'e' b't' b'\r' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [83]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Let the suicide doors up\r\nI threw suicides on the tour bus\r\nI threw suicides on the private jet\r\nYou '
b"know what that mean, I'm fly to death\r\nI step in Def Jam buildin' like I'm the shit\r\nTell 'em give me"
b" fifty million or I'ma quit\r\nMost rappers' taste level ain't at my waist level\r\nTurn up the bass 'til"
b" it's up-in-yo-face level\r\nDon't do no press but I get the most press kit\r\nPlus, yo, my bitch make yo"
b"ur bitch look like Precious\r\nSomethin' 'bout Mary, she gone off that molly\r\nNow the whole party is me"


In [84]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [87]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [88]:
dataset = sequences.map(split_input_target)

In [90]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Let the suicide doors up\r\nI threw suicides on the tour bus\r\nI threw suicides on the private jet\r\nYou'
Target: b'et the suicide doors up\r\nI threw suicides on the tour bus\r\nI threw suicides on the private jet\r\nYou '


In [91]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [92]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [93]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [94]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [95]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 96) # (batch_size, sequence_length, vocab_size)


In [96]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  24576     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  98400     
                                                                 
Total params: 4,061,280
Trainable params: 4,061,280
Non-trainable params: 0
_________________________________________________________________


In [97]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [99]:
sampled_indices

array([45, 84, 88, 41, 17, 78, 24, 66, 82, 43, 13, 24,  9, 19, 32, 55, 84,
       15, 14, 78, 90, 39, 18, 88, 64, 90, 12, 48, 19, 89, 24, 51,  4, 32,
        9, 49, 84,  0, 16, 27, 81, 87, 23, 28, 39, 42, 52, 54, 34, 31, 44,
       32, 83, 93, 29,  0, 16, 42, 55, 25, 11, 92, 25, 80, 57, 61, 51, 64,
       26, 46,  0,  8, 94, 45, 39, 89, 22, 54, 57, 53, 32, 15,  0, 44, 40,
       91, 78, 27, 91, 93, 34, 26, 53, 83, 77, 95, 74, 88, 29,  6],
      dtype=int64)

In [102]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'o exciting\r\n\r\nDinner with Anna Wintour, racing with Anja Rubik\r\nI told you motherfuckers it was more'

Next Char Predictions:
 b"O\xc2\xb7\xc3\xb1K/v6jzM+6'1BY\xc2\xb7-,v\xc4\x81I0\xc3\xb1h\xc4\x81*R1\xc3\xb36U!B'S\xc2\xb7[UNK].9y\xc3\xad5:ILVXDANB~\xe2\x80\x9c;[UNK].LY7)\xe2\x80\x937xaeUh8P[UNK]&\xe2\x80\x9dOI\xc3\xb34XaWB-[UNK]NJ\xe2\x80\x8bv9\xe2\x80\x8b\xe2\x80\x9cD8W~u\xe2\x80\xa6r\xc3\xb1;#"


In [103]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [104]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 96)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.5642743, shape=(), dtype=float32)


In [105]:
tf.exp(example_batch_mean_loss).numpy()

95.992905

In [106]:
model.compile(optimizer='adam', loss=loss)

In [123]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [111]:
EPOCHS = 50

In [112]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
41/41 [==============================] - 80s 2s/step - loss: 3.4116
Epoch 2/50
41/41 [==============================] - 81s 2s/step - loss: 2.5505
Epoch 3/50
41/41 [==============================] - 80s 2s/step - loss: 2.3264
Epoch 4/50
41/41 [==============================] - 82s 2s/step - loss: 2.2178
Epoch 5/50
41/41 [==============================] - 83s 2s/step - loss: 2.1162
Epoch 6/50
41/41 [==============================] - 86s 2s/step - loss: 2.0138
Epoch 7/50
41/41 [==============================] - 82s 2s/step - loss: 1.9223
Epoch 8/50
41/41 [==============================] - 82s 2s/step - loss: 1.8415
Epoch 9/50
41/41 [==============================] - 81s 2s/step - loss: 1.7721
Epoch 10/50
41/41 [==============================] - 81s 2s/step - loss: 1.7081
Epoch 11/50
41/41 [==============================] - 82s 2s/step - loss: 1.6501
Epoch 12/50
41/41 [==============================] - 85s 2s/step - loss: 1.5984
Epoch 13/50
41/41 [==============================

ValueError: Cannot assign value to variable ' my_model/embedding_2/embeddings:0': Shape mismatch.The variable shape (96, 256), and the assigned value shape (88, 256) are incompatible.

In [125]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [126]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [130]:
start = time.time()
states = None
next_char = tf.constant(['Yeezy:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Yeezy: Ye!"
It's finned friends like I'm crazy
Don't ever think that'd phines on 'em
Me and Jor a feel life, n'w things
They was leaning is and I'm looking for the one kers that's hop grount
You know what though? You my father be with you first slow
Try and make it to the party 'fore we finally dog
I feel like one day I need a stylist can't let her fuck it all
When it still look goine it ain't no channels
But it's quite ok for a gangsta to wear sandash
Like killin't the hell you think these bitches comin' at me fo'
But since they all fall in my Palm, I talk my shrestm

Now, I make noise boothers, that's a seaf where the back
Ore we all go
Not just by myself, lookin' for niggas like where's Waldo
She got the ponice, that's how I feel
To be down, you must appeal
That's up to the morgue ne tried to give and just like you
But this is for everything I want now?
Now everybody got through the pain and it mothers
Told Kay, "Don't drunk and hot girl
You want to sit down, huh, st 

_____________